In [20]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import pickle

from gtfparse import read_gtf

# 24/59 genes in [Ivanov et al, 2011] have been annotated in gencode35. 

# What if we remove their annotation and try to predict them using PhyloCSF score? 

In [21]:
# genes from 2011 paper
genes_42 = 'RASL10B,FGFR1,YPEL2,ENOX2,UFSP1,WDR26,TIAL1,YPEL4,NGF,TRPV6,KCTD11,STARD10,YPEL1,R3HCC1,ZFP62,NHLRC4,EPHX3,C1QL4,TLE3,HDGF,ANKRD42,FAM217B,NFKBID,PTEN,RNF187,METTL23,TMEM8B,VANGL2,FNDC5,C1QL1,CITED1,EIF4G3,HELZ2,CYTH2,NTF3,MFSD4B,C1QL3,KCNN4,IFT46,RASD2,CITED2,C1QL2'.split(',')
genes_17 = 'GTF3A, EIF4G2, SP3, PRPS1L1, TEAD4, TEAD3, CACNG8, OAZ3, TEAD1, DDX17, VEGFA, NR1I2, HCK, WT1, BAG1, NPW, MYC'.split(', ')

In [22]:
# open metadata with scores and metrics
meta = pd.read_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t')

metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')
print (metadata_pc_g25.shape)

metadata_pc_g35 = pd.read_csv('tmp_res/metadata_pc_g35.txt', sep='\t')
print (metadata_pc_g35.shape)


# open SET1 (PhyloSET) and SET2 (RiboSET)
SET1 = pd.read_csv('tmp_res/SET1.txt', sep='\t')
SET2 = pd.read_csv('tmp_res/SET2.txt', sep='\t')

(94359, 14)
(101486, 14)


In [23]:
# open file with extension sequences from 2011 paper 
# there is .pdf file in supplementary data -> I took sequences manually from them 
EXT_SEQ_DF = pd.read_csv('data/extensions_from_2011.txt', sep='\t')

In [24]:
genes24 = ['FNDC5','OAZ3','RNF187', 'SP3', 'NR1I2', 'TEAD3',
 'VEGFA', 'PRPS1L1', 'TRPV6', 'R3HCC1', 'MYC', 'BAG1', 'EIF4G2', 
           'WT1', 'TEAD4', 'GTF3A', 'NPW', 'KCTD11', 'YPEL2', 'NFKBID', 'CACNG8', 'HCK', 'YPEL1', 'DDX17']

len(genes24)

24

In [25]:
ann_ext_g35 = pd.read_csv('tmp_res/ANNOTATED_EXT_FOUND_df_g35_pc_complete_CDS.txt', sep='\t')
ann_ext_g35[0:2]

,tr_id,gene,refseq_id,cds_find,nte_find,biotype,tag
0,ENST00000373471.8,FNDC5,NM_153756.3,0,-1,protein_coding,"non_ATG_start,basic,CCDS"
1,ENST00000479764.6,OAZ3,NM_016178.2,0,-1,protein_coding,"non_canonical_other,non_ATG_start,basic,exp_conf"


In [26]:
EXT_SEQ_DF = pd.read_csv('data/extensions_from_2011.txt', sep='\t')

## We are going to extract their nonAUG proteoforms from gencode35 and 'remove' their nonAUG annotation back to prevAUG one (downstream)

interesting that for 2 genes, nonAUG proteoforms are annotated without 5'UTRs:

In [27]:
set(genes24) - set(metadata_pc_g35[(metadata_pc_g35['gene'].isin(genes24)) & 
                (metadata_pc_g35['cds_start_codon'] != 'ATG') & 
                 metadata_pc_g35['5UTR_start_seq'].notna()].gene.tolist())

{'YPEL1', 'YPEL2'}

### plan

* get a position of 1st downstream in-frame AUG (in CDS)
* check whether it's the 1st downstream AUG was annotated before as the start (maybe the next one for some reason?)
* 're-annotate' transcripts -> local coordinates of potential NTE -> global -> PCSF score calculation


In [28]:
df = pd.concat([metadata_pc_g35[(metadata_pc_g35['gene'].isin(genes24)) & 
                (metadata_pc_g35['cds_start_codon'] != 'ATG') & 
                 metadata_pc_g35['5UTR_start_seq'].notna()],
            metadata_pc_g35[metadata_pc_g35['gene'].isin(['YPEL1', 'YPEL2']) &
                           (metadata_pc_g35['cds_start_codon'] != 'ATG')]])

print (df.gene.nunique())

df[0:2]

24


,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
1997,ENST00000373471.8,FNDC5,FNDC5-201,AGTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCA...,AGTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCA...,ACC,ATACACCCCGGGTCGCCGAGCGCCTGGCCGCCCCGCGCCCGCGCCG...,ATA,TGA,166,805,0,166,ENST00000373471.8|ENSG00000160097.19|OTTHUMG00...
1998,ENST00000649537.1,FNDC5,FNDC5-207,CCCCGGGCCTGCCGGCCGGAGGAGCCACCATACACCCCGGGTCGCC...,CCCCGGGCCTGCCGGCCGGAGGAGCCACC,ACC,ATACACCCCGGGTCGCCGAGCGCCTGGCCGCCCCGCGCCCGCGCCG...,ATA,TGA,29,635,0,29,ENST00000649537.1|ENSG00000160097.19|OTTHUMG00...


### iterate over 'CDS': find the 1st ATG 

In [29]:
li = []

for el in df[['tr_id', 'gene', 'cds_start_pos', 'transcript_seq']].to_numpy():
    tr_id = el[0]
    gene = el[1]
    cds_start_pos = el[2]
    tr_seq = el[3]
    
    CDS_seq = tr_seq[cds_start_pos:]
    
    #iterate over CDS seq by 3
    for pos, codon in enumerate([CDS_seq[i:i+3] for i in range(0, len(CDS_seq), 3)]):
        if codon == 'ATG':
            first_dATG_pos = cds_start_pos + pos*3
            
            len_of_real_ext = (first_dATG_pos - cds_start_pos)/3
            
            li.append([tr_id, gene, tr_seq, cds_start_pos, first_dATG_pos, len_of_real_ext])
            
            #print (tr_id, tr_seq[first_dATG_pos:first_dATG_pos+3])
            break 
            
    

In [30]:
dATG_df = pd.DataFrame(li, columns = ['tr_id', 'gene', 'tr_seq', 
                                      'cds_start_pos', 'first_dATG_pos', 'len_of_real_ext_in_codons'])

print (dATG_df.gene.nunique())

# theoretical extension would be [STOP, first_dATG_pos]
dATG_df[0:4]

24


,tr_id,gene,tr_seq,cds_start_pos,first_dATG_pos,len_of_real_ext_in_codons
0,ENST00000373471.8,FNDC5,AGTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCA...,166,391,75.0
1,ENST00000649537.1,FNDC5,CCCCGGGCCTGCCGGCCGGAGGAGCCACCATACACCCCGGGTCGCC...,29,386,119.0
2,ENST00000479764.6,OAZ3,GAAAGTATGGGGAGCGGGCCGGGGAACAGATGGCACCGCGGCTTGC...,3144,3288,48.0
3,ENST00000400999.6,OAZ3,GTTGCCTAAACCTCTGCCACCCACCTGTGAACTTCACTTTGCCACA...,67,211,48.0


In [31]:
def get_pos_of_1st_stop_inframe(seq):
    '''
    seq = seq of 5'leader
    '''
    
    # reverse seq
    seq_r = seq[::-1]
    
    i = 0  
    li = []
    while i <= len(seq_r):    
    
        codon = seq_r[i:i+3][::-1] # read by codon and reverse it like it's read on forward strand
        if (codon == 'TAG') | (codon == 'TGA') | (codon == 'TAA'):
            li.append(i) 
        i = i + 3
        
    if li:
        pos = (np.min(li))
        pos_f = len(seq) - pos
    else:
        # check triplets 
        if len(seq) % 3 == 0:
            pos_f = 0
        elif len(seq) % 3 == 1:
            pos_f = 1
        else:
            pos_f = 2
    
    return (pos_f)

In [32]:
dATG_df.columns

Index(['tr_id', 'gene', 'tr_seq', 'cds_start_pos', 'first_dATG_pos',
       'len_of_real_ext_in_codons'],
      dtype='object')

In [33]:
# add the last 3'in-frame STOP
li = []

for el in dATG_df.to_numpy():
    tr_id = el[0]
    gene = el[1]
    cds_start_pos = el[3]
    tr_seq = el[2]
    first_dATG_pos = el[4]
    len_of_real_ext_in_codons = el[-1]
    
    utr5_seq = tr_seq[0:first_dATG_pos]
    
    pos_stop = get_pos_of_1st_stop_inframe(utr5_seq)
    
    nte_seq = tr_seq[pos_stop:first_dATG_pos]
    
    nte_ext_len = len(nte_seq) / 3
    
    li.append([tr_id, gene, tr_seq, first_dATG_pos, cds_start_pos, utr5_seq, 
               pos_stop, nte_seq, nte_ext_len, len_of_real_ext_in_codons])
    
nte_local_df = pd.DataFrame(li, columns = ['tr_id', 'gene', 'tr_seq', 'new_N_term_end',
                                          'cds_start_pos', 'new_utr5_seq', 'new_N_term_start',
                                          'new_NTE_seq', 'new_NTE_len_codons', 'len_of_real_ext_in_codons'])

print (nte_local_df.gene.nunique())

nte_local_df[0:2]

24


,tr_id,gene,tr_seq,new_N_term_end,cds_start_pos,new_utr5_seq,new_N_term_start,new_NTE_seq,new_NTE_len_codons,len_of_real_ext_in_codons
0,ENST00000373471.8,FNDC5,AGTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCA...,391,166,AGTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCA...,1,GTCGCTGTCCGCGGAGCCGACATGCAGGCGGCTCGGGGCGGCGCAG...,130.0,75.0
1,ENST00000649537.1,FNDC5,CCCCGGGCCTGCCGGCCGGAGGAGCCACCATACACCCCGGGTCGCC...,386,29,CCCCGGGCCTGCCGGCCGGAGGAGCCACCATACACCCCGGGTCGCC...,2,CCGGGCCTGCCGGCCGGAGGAGCCACCATACACCCCGGGTCGCCGA...,128.0,119.0


## add 50-codons long or less local coordinate 

In [34]:
def add_new_NTE_start_50_codons(x):
    '''
    return N_term_start_50
    if there are >50 codons = all 50
    if less = all we have 
    '''
    if x['new_NTE_len_codons'] >= 50:
        New_nte_start = x['new_N_term_end'] - 150  
    else:
        New_nte_start = x['new_N_term_start']
        
    return New_nte_start
        

nte_local_df['50len_flag'] = nte_local_df.apply(add_new_NTE_start_50_codons, axis=1)

In [35]:
nte_local_df[['50len_flag', 'new_N_term_start', 'new_N_term_end']][0:4]

,50len_flag,new_N_term_start,new_N_term_end
0,241,1,391
1,236,2,386
2,3138,3135,3288
3,61,58,211


In [36]:
nte_local_df.to_csv('tmp_res/reprod_24genes_2011_local_coo_df.txt', sep='\t', index=False)

# Open global coo file and parse 

In [37]:
def prepare_global_coo(path_to_file_with_global_coo, colname):
    global_coo_g25 = pd.read_csv(path_to_file_with_global_coo, sep='\t')
    global_coo_g25_f = global_coo_g25[global_coo_g25['hit'] == True]
    global_coo_g25_f = global_coo_g25_f.sort_values(by=['seqnames', 'start'])
    global_coo_g25_f['global_coo'] = global_coo_g25_f['seqnames']+':'+global_coo_g25_f['start'].astype(str) +'-'+global_coo_g25_f['end'].astype(str)
    global_coo_g25_f = global_coo_g25_f[['group_name', 'strand', 'global_coo']].groupby(['group_name', 'strand']).agg('+'.join).reset_index()
    global_coo_g25_f.columns = ['tr_id', 'strand', colname]
    return global_coo_g25_f

In [38]:
codons50_global = prepare_global_coo(path_to_file_with_global_coo='tmp_res/reprod_24genes_2011_global_coo_df.txt',
                                    colname = 'global_coo_50_and_less')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [39]:
# merge it to local df

nte_local_global_df = nte_local_df.merge(codons50_global, on='tr_id', how='inner')

print (nte_local_global_df.shape, nte_local_global_df.gene.nunique())

nte_local_global_df.to_csv('tmp_res/reprod_24genes_2011_local_global_coo_df.txt', sep='\t', index=False)

(60, 13) 24


In [40]:
# send it to server to calculated 50-codons PCSF score 

# Parse PhyloCSF results 

In [41]:
li = []

for path in glob.glob('data/PCSF_24genes/*/*.txt'):
    try: 
        tmp = pd.read_csv(path, sep='\t', header=None)
        li.append(tmp)
        
    except Exception as e:     
        with open(path, 'r') as f:
            li1 = []
            for l in f.readlines():
                if (l.find('Failure("invalid MFA alignment: Invalid_argument(\\"Option.get\\")")') == -1) | (l.find('Failure("no regions successfully evaluated")') == -1):
                    li1.append(l.strip().split('\t'))
                else:
                    print (l.split('\t')[0])
            tmp2 = pd.DataFrame(li1)
            li.append(tmp2)

PhyloCSF_50 = pd.concat(li)[[0, 1, 2]]

PhyloCSF_50['tr_id'] = [x.split('/')[-1].split('.fasta')[0] for x in PhyloCSF_50[0].tolist()]

PhyloCSF_50 = PhyloCSF_50[(PhyloCSF_50[2] != 'Failure("no regions successfully evaluated")') & 
                          (PhyloCSF_50[2] != 'Failure("invalid MFA alignment: Invalid_argument(\\"Option.get\\")")') & 
                          (PhyloCSF_50[1] != 'abort')]


PhyloCSF_50 = PhyloCSF_50[['tr_id', 2]] 


PhyloCSF_50[2] = PhyloCSF_50[2].astype(float)

PhyloCSF_50.columns = ['tr_id', 'PhyloCSF120score']

PhyloCSF_50['tr_id1'] = [x.split('.')[0] for x in PhyloCSF_50['tr_id'].tolist()]

In [42]:
PhyloCSF_50[0:2]

,tr_id,PhyloCSF120score,tr_id1
0,ENST00000634734.2,-1026.5729,ENST00000634734
0,ENST00000396821.7,421.9530,ENST00000396821


In [43]:
nte_local_global_df[nte_local_global_df['tr_id'].isin(ann_ext_g35.tr_id.tolist())].gene.nunique()

24

In [45]:
nte_local_global_PCSF_df = nte_local_global_df.merge(PhyloCSF_50,  on=['tr_id'], how='left')
nte_local_global_PCSF_df.columns

Index(['tr_id', 'gene', 'tr_seq', 'new_N_term_end', 'cds_start_pos',
       'new_utr5_seq', 'new_N_term_start', 'new_NTE_seq', 'new_NTE_len_codons',
       'len_of_real_ext_in_codons', '50len_flag', 'strand',
       'global_coo_50_and_less', 'PhyloCSF120score', 'tr_id1'],
      dtype='object')

In [46]:
nte_local_global_PCSF_df[nte_local_global_PCSF_df['PhyloCSF120score'] > 0].gene.nunique()

11

In [48]:
nte_local_global_PCSF_df.to_csv('tmp_res/PCSF_for_24_ann.txt', sep='\t', index=False)

### these transcripts contain NTE-sequence predicted in [2011]

In [27]:
nte_local_global_df[(nte_local_global_df['tr_id'].isin(ann_ext_g35.tr_id.tolist())) 
                   & (nte_local_global_PCSF_df['PhyloCSF120score'] > 0)].gene.nunique()

11

In [28]:
nte_local_global_PCSF_df[(nte_local_global_PCSF_df['tr_id'].isin(ann_ext_g35.tr_id.tolist()))][['tr_id', 'gene', 'global_coo_50_and_less', 
                                                                                      'strand', 'PhyloCSF120score', 'len_of_real_ext_in_codons']]

,tr_id,gene,global_coo_50_and_less,strand,PhyloCSF120score,len_of_real_ext_in_codons
0,ENST00000373471.8,FNDC5,chr1:32868374-32868388+chr1:32868882-32868997+...,-,2051.0659,75.0
2,ENST00000479764.6,OAZ3,chr1:151766716-151766865,+,-139.7141,48.0
3,ENST00000400999.6,OAZ3,chr1:151766716-151766865,+,-139.7141,48.0
4,ENST00000305943.8,RNF187,chr1:228487666-228487815,+,-742.6165,109.0
5,ENST00000418194.6,SP3,chr2:173955657-173955806,-,350.1512,217.0
6,ENST00000652005.1,SP3,chr2:173955657-173955806,-,350.1512,217.0
7,ENST00000393716.6,NR1I2,chr3:119807266-119807415,+,-625.4103,55.0
8,ENST00000466380.5,NR1I2,chr3:119807266-119807415,+,-625.4103,55.0
10,ENST00000338863.12,TEAD3,chr6:35486468-35486617,-,1626.5325,65.0
11,ENST00000639578.2,TEAD3,chr6:35486468-35486617,-,1626.5325,65.0


### Remaining genes 

In [85]:
remaining_genes = nte_local_global_PCSF_df[(~nte_local_global_PCSF_df['gene'].isin(list(nte_local_global_PCSF_df[nte_local_global_PCSF_df['PhyloCSF120score'] > 0].gene.unique()))) &
                                          (nte_local_global_df['tr_id'].isin(ann_ext_g35.tr_id.tolist()))]


print (remaining_genes.gene.nunique())

13


In [128]:


#remaining_genes[['tr_id', 'gene', 'global_coo_50_and_less',
                 #'strand', 'PhyloCSF120score', 'len_of_real_ext_in_codons']]
    
i = 7

print ('gene, tr_id', remaining_genes.iloc[i].gene, remaining_genes.iloc[i].tr_id)

print ('coo: ', remaining_genes.iloc[i].global_coo_50_and_less, remaining_genes.iloc[i].strand)

print ()

print ('real ext len', remaining_genes.iloc[i].len_of_real_ext_in_codons)

print ()

print (remaining_genes.iloc[i].PhyloCSF120score)

print ()

print ('theor.nte-seq', nte_local_global_PCSF_df[nte_local_global_PCSF_df['tr_id'] == remaining_genes.iloc[i].tr_id].iloc[0].new_NTE_seq)

print ()

print ('ext pred in paper')
print (EXT_SEQ_DF[EXT_SEQ_DF['gene'] == remaining_genes.iloc[i].gene].iloc[0].ext_seq[:-3])

gene, tr_id TRPV6 ENST00000359396.8
coo:  chr7:142885517-142885666 -

real ext len 40.0

-1512.0518

theor.nte-seq GAGTCCTGGCTGGCTCTGCCAAGTGTAACAAACTCACAGCCCTCTCCAAACTGGCTGGGGCTGCTGGGAGACTCCCAAGGAACTCGTCAGGAAGGCAGGAGACAGGAGACGGGACCTCTACAGGGAGACGGTGGGCCGGCCCTTGGGGGGGCTGATGTGGCCCCAAGGCTGAGTCCCGTCAGGGTCTGGCCTCGGCCTCAGGCCCCCAAGGAGCCGGCCCTACACCCC

ext pred in paper
ACGGGACCTCTACAGGGAGACGGTGGGCCGGCCCTTGGGGGGGCTGATGTGGCCCCAAGGCTGAGTCCCGTCAGGGTCTGGCCTCGGCCTCAGGCCCCCAAGGAGCCGGCCCTACACCCC


In [108]:
remaining_genes[['tr_id', 'gene', 'global_coo_50_and_less', 'strand', 'PhyloCSF120score', 'len_of_real_ext_in_codons']]

,tr_id,gene,global_coo_50_and_less,strand,PhyloCSF120score,len_of_real_ext_in_codons
2,ENST00000479764.6,OAZ3,chr1:151766716-151766865,+,-139.7141,48.0
3,ENST00000400999.6,OAZ3,chr1:151766716-151766865,+,-139.7141,48.0
4,ENST00000305943.8,RNF187,chr1:228487666-228487815,+,-742.6165,109.0
7,ENST00000393716.6,NR1I2,chr3:119807266-119807415,+,-625.4103,55.0
8,ENST00000466380.5,NR1I2,chr3:119807266-119807415,+,-625.4103,55.0
12,ENST00000372067.7,VEGFA,chr6:43771097-43771246,+,-708.6328,180.0
13,ENST00000672860.2,VEGFA,chr6:43771097-43771246,+,-708.6328,180.0
15,ENST00000359396.8,TRPV6,chr7:142885517-142885666,-,-1512.0518,40.0
18,ENST00000265806.11,R3HCC1,chr8:23290029-23290178,+,-885.1802,187.0
24,ENST00000621592.7,MYC,chr8:127736591-127736623+chr8:127738248-127738262,+,-164.9417,15.0
